In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


from scipy import stats
from sklearn.metrics import r2_score,mean_absolute_error
import xgboost as xgb
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score,mean_squared_error


In [2]:
df = pd.read_csv("divyanshu/PreCog/data/judge/judges_clean.csv")

In [3]:
df.head()

,ddl_judge_id,state_code,dist_code,court_no,judge_position,female_judge,start_date,end_date
0,1,1,1,1,chief judicial magistrate,0 nonfemale,20-09-2013,20-02-2014
1,2,1,1,1,chief judicial magistrate,0 nonfemale,31-10-2013,20-02-2014
2,3,1,1,1,chief judicial magistrate,0 nonfemale,21-02-2014,31-05-2016
3,4,1,1,1,chief judicial magistrate,0 nonfemale,01-06-2016,06-06-2016
4,5,1,1,1,chief judicial magistrate,0 nonfemale,06-06-2016,07-07-2018


In [4]:
df.state_code.nunique()

29

In [5]:
df.dist_code.nunique()

74

In [6]:
df.female_judge.nunique()

3

In [7]:
df.judge_position.nunique()

565

In [8]:
from sklearn.preprocessing import LabelEncoder
    
# creating instance of labelencoder
encoder1 = LabelEncoder()
encoder2 = LabelEncoder()
    
df['jp_encoded'] = encoder1.fit_transform(df['judge_position'])
df['target'] = encoder2.fit_transform(df['female_judge'])

In [9]:
df.shape

(98478, 10)

In [10]:
df.drop(['ddl_judge_id','start_date','end_date','judge_position','female_judge'],axis=1,inplace=True)

In [11]:
X = df.drop('target', axis=1)
y = df['target']

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_comb,Y_train,Y_comb = train_test_split(X,y,test_size=0.3,random_state=0 , shuffle = False)
X_validation,X_test,Y_validation,Y_test = train_test_split(X_comb,Y_comb,test_size=0.5,random_state=0 , shuffle = False)

In [13]:
X_train.shape,Y_train.shape

((68934, 4), (68934,))

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68934 entries, 0 to 68933
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   state_code  68934 non-null  int64
 1   dist_code   68934 non-null  int64
 2   court_no    68934 non-null  int64
 3   jp_encoded  68934 non-null  int64
dtypes: int64(4)
memory usage: 2.6 MB


In [15]:
xgb_model = xgb.XGBClassifier().fit(X_train, Y_train)

In [16]:
y_pred_val = xgb_model.predict(X_validation)
y_pred_test = xgb_model.predict(X_test)

In [17]:
print('Accuracy of XGB regressor on training set: {:.2f}'
       .format(xgb_model.score(X_train, Y_train)))
print('Accuracy of XGB regressor on validation set: {:.2f}'
       .format(xgb_model.score(X_validation, Y_validation)))
print('Accuracy of XGB regressor on test set: {:.2f}'
       .format(xgb_model.score(X_test, Y_test)))
print("f1 Score on validation set: {:.2f}"
       .format(f1_score(Y_validation, y_pred_val, average='weighted')))
print("f1 Score on test set: {:.2f}"
       .format(f1_score(Y_test, y_pred_test, average='weighted')))
print("Mean Squared Error on test: {:.2f}"
       .format(mean_squared_error(Y_test, y_pred_test, squared=False)))


Accuracy of XGB regressor on training set: 0.75
Accuracy of XGB regressor on validation set: 0.54
Accuracy of XGB regressor on test set: 0.66
f1 Score on validation set: 0.40
f1 Score on test set: 0.56
Mean Squared Error on test: 0.58
